In [1]:
import os
import SimpleITK as sitk
import numpy as np
import pandas as pd


In [2]:
# Function to compute LBP from NIfTI file
def compute_lbp_2_from_nifti(file_path):
    # Read NIfTI file using SimpleITK
    image = sitk.ReadImage(file_path)
    image_array = sitk.GetArrayFromImage(image)

    # Calculate LBP using the provided code snippet
    result = np.zeros_like(image_array, dtype=np.uint8)

    result[1:-1, 0:-2] |= (image_array[0:-2, 1:-1] > image_array[1:-1, 1:-1])
    result[1:-1, 0:-2] |= np.left_shift((image_array[1:-1, 2:] > image_array[1:-1, 1:-1]), 1).astype(np.uint8)
    # Continue similar operations for the other positions...
     # Perform similar operations for the other 6 positions
    result[1:-1, 2:] |= np.left_shift((image_array[0:-2, 2:] > image_array[1:-1, 1:-1]), 2).astype(np.uint8)
    result[1:-1, 2:] |= np.left_shift((image_array[0:-2, 1:-1] > image_array[1:-1, 1:-1]), 3).astype(np.uint8)
    
    result[2:, 2:] |= np.left_shift((image_array[0:-2, 2:] > image_array[1:-1, 1:-1]), 4).astype(np.uint8)
    result[2:, 2:] |= np.left_shift((image_array[1:-1, 2:] > image_array[1:-1, 1:-1]), 5).astype(np.uint8)
    
    result[2:, 1:-1] |= np.left_shift((image_array[0:-2, 0:-2] > image_array[1:-1, 1:-1]), 6).astype(np.uint8)
    result[2:, 1:-1] |= np.left_shift((image_array[1:-1, 0:-2] > image_array[1:-1, 1:-1]), 7).astype(np.uint8)
    
    result[2:, 0:-2] |= np.left_shift((image_array[0:-2, 0:-2] > image_array[1:-1, 1:-1]), 0).astype(np.uint8)
    result[2:, 0:-2] |= np.left_shift((image_array[1:-1, 0:-2] > image_array[1:-1, 1:-1]), 1).astype(np.uint8)

    return result

In [3]:
# Function to calculate texture properties from LBP matrix
def calculate_texture_properties(lbp_matrix):
    # Calculate texture properties (replace with your implementation)
    # Here's a dummy implementation for demonstration purposes
    texture_properties = {
        'Contrast': np.mean(lbp_matrix),
        'Dissimilarity': np.std(lbp_matrix),
        'Homogeneity': np.sum(lbp_matrix) / (np.max(lbp_matrix) + 1),
        'Correlation': np.corrcoef(lbp_matrix.ravel(), lbp_matrix.ravel())[0, 1],
        'Energy': np.sum(lbp_matrix ** 2) / lbp_matrix.size
    }
    return texture_properties


In [4]:
# Path to directory containing all NIfTI files
data_dir = 'F:\Image CHD\ImageCHD_dataset\Training Data'

In [5]:
# Initialize an empty DataFrame to store results
results_df = pd.DataFrame(columns=['File', 'Contrast', 'Dissimilarity', 'Homogeneity', 'Correlation', 'Energy'])

In [6]:
# Loop through each file in the directory
for file_name in os.listdir(data_dir):
    if file_name.endswith('.nii.gz'):
        # Compute LBP from NIfTI file
        nifti_file_path = os.path.join(data_dir, file_name)
        lbp_result = compute_lbp_2_from_nifti(nifti_file_path)
        
        # Calculate texture properties from LBP result
        texture_properties = calculate_texture_properties(lbp_result)
        
        # Add file name and properties to DataFrame
        results_df.loc[len(results_df)] = [file_name, *texture_properties.values()]


In [7]:
# Save results to Excel file
results_df.to_excel('LBP_propertie.xlsx', index=False)